In [1]:
import pandas as pd
from math import isnan

In [2]:
GT_plates = pd.read_excel("PlatesLateral.ods", engine="odf")
GT_plates_videos = pd.read_excel("PlatesVideos.ods", engine="odf")
results = pd.read_csv('../out/results_confosable_Bidi.csv')
results_videos = pd.read_csv('../out/results_videos.csv')
results_times_gpu = pd.read_csv('../out/results_times_videos_GPU.csv')
#results_times_cpu = pd.read_csv('../out/results_videos.csv')
GT_plates = GT_plates[GT_plates['Plate'].notna()]
GT_plates['Plate'] = GT_plates['Plate'].apply(lambda x: str(x).replace('“','').replace('”',''))
GT_plates

,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,Y
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [3]:
results

,Model,Video,Frame,Pred_plate,Confidence
0,a109,a109e_takeoff,0,NaN,0.000000
1,a109,a109e_takeoff,1,NaN,0.000000
2,a109,a109e_takeoff,2,405,0.680593
3,a109,a109e_takeoff,3,NaN,0.000000
4,a109,a109e_takeoff,4,NaN,0.000000
...,...,...,...,...,...
4097,w3,Saving,102,NaN,0.000000
4098,w3,Saving,103,NaN,0.000000
4099,w3,Saving,104,NaN,0.000000
4100,w3,Saving,105,NaN,0.000000


In [4]:
results_videos

,Model,Video,Frame,Pred_plate,Confidence
0,a109,corporate,0,W2SHDT,0.999998
1,a109,corporate,1,N2&SHD,1.000000
2,a109,corporate,2,M2SHD,0.999951
3,a109,corporate,3,N2UHD,1.000000
4,a109,corporate,4,NAA,0.021489
...,...,...,...,...,...
76441,w3,Rare,325,$0,0.854689
76442,w3,Rare,326,0,0.630538
76443,w3,Rare,327,1U,0.972253
76444,w3,Rare,328,SImS,0.988457


In [5]:
# Analyze results from images

In [6]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results.loc[(results['Model'] == row['Model']) & (results['Video'] == row['Video'])]
    #If a frame interval is defined, get only plates in that intervals
    if not isnan(row['Init_frame']):
        preds = preds[preds['Frame'].isin(range(int(row['Init_frame']),int(row['End_frame']+1)))]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

0.6058394160583942 0.5036496350364964


,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,Y
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [7]:
# Analyze results from videos

In [8]:
n = GT_plates_videos.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates_videos.iterrows():
    #Get plates recognized in that video
    preds = results_videos.loc[(results_videos['Model'] == row['Model']) & (results_videos['Video'] == row['Video'])]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        #print(preds['Pred_plate'].mode(dropna=True))
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates_videos['Detected'] = detected
GT_plates_videos.to_excel("PlatesVideos.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates_videos

0.6614173228346457 0.25196850393700787


,Model,Video,Plate,Detected
0,a109,a109e_takeoff,JA6801,N
1,a109,augusta_a109,D-HAVN,Y
2,a109,aw109sp,S2-AJP,Y
3,a109,corporate,N433AK,Y
4,a109,crashed,N109EX,N
...,...,...,...,...
122,uh1,huey,VH-UYH,Y
123,uh1,uh1,N617WC,N
124,w3,EHS,0718,Y
125,w3,Rare,SN-33XG,Y


In [ ]:
#Analyze times with GPU

In [15]:
results_times_gpu

,FOTS,Classifier,Refinement,Detections,Width,Height
0,0.048423,0.002404,0.000214,4,1280,704
1,0.041921,0.000630,0.000057,4,1280,704
2,0.044439,0.000619,0.000065,5,1280,704
3,0.053288,0.000636,0.000049,8,1280,704
4,0.046623,0.000626,0.000037,6,1280,704
...,...,...,...,...,...,...
76441,0.039114,0.000542,0.000033,4,1280,704
76442,0.044552,0.000544,0.000032,5,1280,704
76443,0.044751,0.000549,0.000035,6,1280,704
76444,0.051153,0.000571,0.000049,8,1280,704


In [13]:
results_times_gpu['Height'].value_counts()

1280    69247
480      2447
1600     1910
640      1479
832      1363
Name: Width, dtype: int64

In [14]:
results_times_gpu['Width'].value_counts()

1280    69247
480      2447
1600     1910
640      1479
832      1363
Name: Width, dtype: int64

In [16]:
results_times_gpu['Detections'].value_counts()

2      11110
3      10408
1      10045
4       9366
5       7370
       ...  
177        1
171        1
107        1
164        1
156        1
Name: Detections, Length: 174, dtype: int64

In [20]:
grouped_df = results_times_gpu.groupby('Detections')

mean_df = grouped_df.mean()

mean_df = mean_df.reset_index()

sd_df = grouped_df.std()

sd_df = sd_df.reset_index()

print(mean_df)
print(sd_df)

     Detections      FOTS  Classifier    Refinement        Width      Height
0             0  0.019221    0.000012  1.982028e-07   962.839501  553.800859
1             1  0.028529    0.000536  1.276249e-03  1185.838128  657.916177
2             2  0.033829    0.000559  1.014350e-03  1260.120252  693.426463
3             3  0.037515    0.000576  2.087271e-03  1277.106841  702.345888
4             4  0.040753    0.000591  3.771600e-03  1280.386077  703.825753
..          ...       ...         ...           ...          ...         ...
169         181  0.633229    0.001471  1.290560e-03  1280.000000  704.000000
170         183  0.645212    0.001469  4.053116e-05  1280.000000  704.000000
171         184  0.639307    0.001452  4.429460e-03  1280.000000  704.000000
172         187  0.658110    0.001465  4.935265e-05  1280.000000  704.000000
173         206  0.719687    0.001535  4.241467e-04  1280.000000  704.000000

[174 rows x 6 columns]
     Detections      FOTS    Classifier    Refinemen